In [1]:
def seed_everything(seed: int):
    import os
    import random

    import numpy as np
    import torch

    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(1)

In [2]:
import glob
import os
import re

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import scienceplots
import torch
from fkan.torch import FractionalJacobiNeuralBlock as fJNB
from mpl_toolkits.mplot3d import Axes3D
from torch import nn, optim

In [3]:
plt.style.use("science")
mpl.use("pgf")

plt.rcParams.update(
    {"text.usetex": True, "pgf.preamble": r"\usepackage{amssymb} \usepackage{amsmath}"}
)

In [4]:
def dy_dx(y, x):
    return torch.autograd.grad(
        y, x, grad_outputs=torch.ones_like(y), create_graph=True
    )[0]

In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.jacobies = []
        self.hiddens = []
        self.n_hidden = 2
        for i in range(1, 7):
            act = fJNB(i)
            # act = nn.Tanh()
            self.jacobies.append(act)
            self.hiddens.append(nn.Linear(2, self.n_hidden))
        self.output = nn.Linear(self.n_hidden * len(self.jacobies), 1)

    def forward(self, x):
        acts = []
        for hidden, jacobi in zip(self.hiddens, self.jacobies):
            q = hidden(x)
            acts.append(jacobi(q))
        h = torch.cat(acts, dim=1)
        output = self.output(h)
        return output

In [6]:
x = torch.linspace(0, 1, 100, requires_grad=True)
t = torch.linspace(0, 1, 100, requires_grad=True)

x, t = torch.meshgrid(x, t, indexing="ij")
x = x.reshape(-1, 1)
t = t.reshape(-1, 1)


mlp = Model()
optimizer = optim.LBFGS(list(mlp.parameters()), lr=0.1)

In [7]:
# c, alpha, nu = 0.1, 1, 0.01  # param set 1 (see paper for details)
c, alpha, nu = 0.5, 0.1, 0.0001     # param set 2 (see paper for details)

Exact = lambda x, t: c / alpha + (2 * nu / alpha) * torch.tanh(x - c * t)

In [8]:
def get_loss(x, t):
    x_t = torch.cat((x, t), 1)
    x_0 = torch.cat((x, 0 * t), 1)

    u = mlp.forward(x_t)
    u_t = dy_dx(u, t)
    u_x = dy_dx(u, x)
    u_xx = dy_dx(u_x, x)

    residual_pde = u_t + alpha * u_x * u - nu * u_xx

    residual_init = mlp.forward(x_0) - Exact(x, 0)

    loss = 1e6 * ((residual_pde**2).mean() + (residual_init**2).mean())
    return loss

In [9]:
def closure():
    loss = get_loss(x, t)
    optimizer.zero_grad()
    loss.backward()
    return loss

In [10]:
losses = []

for i in range(1, 100):
    loss = get_loss(x, t)
    optimizer.step(closure)
    losses.append(loss.detach().numpy())
    if i % 2 == 0:
        print("Epoch %3d: Current loss: %.2e" % (i, losses[-1]))

Epoch   2: Current loss: 5.72e+05
Epoch   4: Current loss: 5.98e+03
Epoch   6: Current loss: 1.27e+02
Epoch   8: Current loss: 3.50e+01
Epoch  10: Current loss: 6.65e+00
Epoch  12: Current loss: 5.89e+00
Epoch  14: Current loss: 2.30e-01
Epoch  16: Current loss: 2.12e-01
Epoch  18: Current loss: 2.12e-01
Epoch  20: Current loss: 1.62e-01
Epoch  22: Current loss: 1.20e-01
Epoch  24: Current loss: 1.20e-01
Epoch  26: Current loss: 1.20e-01
Epoch  28: Current loss: 1.20e-01
Epoch  30: Current loss: 1.20e-01
Epoch  32: Current loss: 1.20e-01
Epoch  34: Current loss: 1.20e-01
Epoch  36: Current loss: 1.20e-01
Epoch  38: Current loss: 1.20e-01
Epoch  40: Current loss: 1.20e-01
Epoch  42: Current loss: 1.19e-01
Epoch  44: Current loss: 1.17e-01
Epoch  46: Current loss: 4.14e-03
Epoch  48: Current loss: 6.31e-04


In [11]:
Nx, Nt = 21, 15
x = torch.linspace(0, 1, Nx)
t = torch.linspace(0, 1, Nt)

x, t = torch.meshgrid(x, t, indexing="ij")
x = x.reshape(-1, 1)
t = t.reshape(-1, 1)
x_t = torch.cat((x, t), 1)

exact = Exact(x, t).reshape(Nx, Nt)
predict = mlp.forward(x_t).reshape(Nx, Nt)
error = exact - predict

MAE = torch.abs(error).mean()

print("Mean Absolute Error: %.2e" % MAE)

Mean Absolute Error: 1.17e-05


In [12]:
x = x.reshape(Nx, Nt)
t = t.reshape(Nx, Nt)

fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(131, projection="3d")
ax.plot_surface(x, t, predict.detach().numpy(), cmap="viridis")
ax.view_init(10, 45)

ax = fig.add_subplot(132, projection="3d")
ax.plot_surface(x, t, exact.detach().numpy(), cmap="viridis")
ax.view_init(10, 45)
ax = fig.add_subplot(133, projection="3d")
ax.plot_surface(x, t, error.detach().numpy(), cmap="viridis")
ax.view_init(10, 45)
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(111)
ax.plot(np.log10(losses), "c", label="Loss")

In [13]:
fig = plt.figure(figsize=(12, 7))

ax = fig.add_subplot(111, projection="3d")

ax.plot_surface(x, t, predict.detach().numpy(), cmap="viridis")

ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

ax.xaxis.pane.set_edgecolor("w")
ax.yaxis.pane.set_edgecolor("w")
ax.zaxis.pane.set_edgecolor("w")
ax.set_yticks([0, 0.5, 1])
ax.set_xlabel(r"$\zeta$")
ax.set_ylabel(r"$\tau$")
ax.set_zlabel(r"$\hat{\chi}(\zeta,\tau)$")
ax.view_init(elev=15, azim=-260)

# fig.subplots_adjust(left=0, right=1, top=2, bottom=0)
plt.savefig(
    "burgers-2-prediction.pdf",
    bbox_inches="tight",
    pad_inches=0,
)

fig = plt.figure(figsize=(6, 7))
ax = fig.add_subplot(111, projection="3d")

ax.plot_surface(x, t, error.detach().numpy(), cmap="viridis")

ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

ax.xaxis.pane.set_edgecolor("w")
ax.yaxis.pane.set_edgecolor("w")
ax.zaxis.pane.set_edgecolor("w")
ax.set_yticks([0, 0.5, 1])
ax.set_xlabel(r"$\zeta$")
ax.set_ylabel(r"$\tau$")
ax.set_zlabel(r"$\mathfrak{R}(\zeta,\tau)$")
ax.view_init(elev=15, azim=-260)


ax.zaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax.zaxis.get_major_formatter().set_scientific(True)
ax.zaxis.get_major_formatter().set_powerlimits((0, 0))
plt.savefig(
    "burgers-2-residual.pdf",
    bbox_inches="tight",
    pad_inches=0,
)